In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df


current_con = con
current_df = site_df
current_dm = site_dm
# replace NaN with None
current_dm = current_dm.where((pd.notnull(current_dm)), None)
cols = current_df.columns


#vocab.get_controlled_vocabularies()

# mess up some validations for locations
#current_df.loc['Tel Hazor']['lat_s'] = 400.
#current_df['dir_inc'] = 5
#current_con.tables.pop('sites')
current_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,age,age_high,age_low,age_unit,citations,description,geologic_classes,geologic_types,lat,lithologies,location,lon,site
site,,,,,,,,,,,,,
hz05,-740,-732,-800,Years Cal AD (+/-),This study,Tel Hazor startum 5,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz05
hz06,-740,-732,-800,Years Cal AD (+/-),This study,Tel Hazor startum 6,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz06
hz07,-850,-800,-900,Years Cal AD (+/-),This study,Tel Hazor startum 7,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz07
hz09,-950,-900,-1000,Years Cal AD (+/-),This study,Tel Hazor startum 9,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz09
hz10,-950,-900,-1000,Years Cal AD (+/-),This study,Tel Hazor startum 10,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz10


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()

vocabulary

-I- Importing controlled vocabularies from https://earthref.org


age_unit                 [Ga, Ka, Ma, Years AD (+/-), Years BP, Years C...
alteration                 [Altered, High, Mild, Severe, Trace, Unaltered]
alteration_type          [Acid Leaching, Acid Oxidation, Acid Sulphate,...
aniso_s_unit             [SI, Am^2, bulk in measurements table, Normali...
aniso_type                                         [AMS, AARM, ATRM, AIRM]
assemblage               [Aggregate, In Situ, Mineral Separate, Polycry...
mineral_assemblage       [Aggregate, In Situ, Mineral Separate, Polycry...
int_scat                                  [True, False, true, false, 0, 1]
geologic_classes         [Archeologic, Extraterrestrial, Extrusive, Ign...
conglomerate_test                 [+, -, G+, G-, Go, IG+, IG-, IGo, ND, o]
contact_test                      [+, -, C+, C-, Co, IC+, IC-, ICo, ND, o]
continent_ocean          [Africa, Antarctica, Artic Ocean, Asia, Atlant...
country                  [Afghanistan, Albania, Algeria, American Samoa...
int_corr                 

In [4]:
current_dm[['validations']].dropna()

,validations
age,"[requiredUnless(""age_low"",""age_high""), require..."
age_high,"[min(""age_low""), requiredUnless(""age"")]"
age_low,"[max(""age_high""), requiredUnless(""age"")]"
age_sigma,[min(0)]
age_unit,"[cv(""age_unit""), required()]"
analysts,"[type(""users"")]"
aniso_perc,"[min(0), max(100)]"
aniso_tilt_correction,"[min(-3), max(100), requiredIfGroup(""Anisotrop..."
aniso_total,"[min(0), max(300)]"
aniso_type,"[requiredIfGroup(""Anisotropy"")]"


In [5]:
# check column validity
invalid_cols = [col for col in cols if col not in current_dm.index]


def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return True


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return True
    elif table_name in current_con.tables:
        return True
    else:
        print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)
    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return True
        else:
            print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return True


def required(col_name, df, arg):
    #print df
    if col_name in df.columns:
        return True
    else:
        return False
    

def check_max(row, col_name, arg, *args):
    #print 'check_max'
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    arg = float(arg)
    if float(cell_value) <= float(arg):
        return True
    else:
        return "{} must be <= {}".format(str(cell_value), str(arg))

def check_min(row, col_name, arg, *args):
    #print 'check_min'
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    if float(cell_value) >= float(arg):
        return True
    else:
        return "{} must be >= {}".format(str(cell_value), str(arg))

def cv(row, col_name, arg, current_data_model):
    #print 'cv', col_name, arg
    #cv geologic_classes class
    cell_value = row[col_name]
    #print cell_value
    #print 'vocabulary[arg]', vocabulary[col_name]
    if cell_value.lower() in vocabulary[col_name]:
        print 'returning True'
        return True
    else:
        print 'returning ', "{} is not in {}".format(cell_value, arg)
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": check_max, "min": check_min, "cv": cv}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


for validation_name, validation in current_dm.iterrows():
    val_list = validation['validations']
    if not val_list:
        continue
    for num, val in enumerate(val_list):
        # split val into function & arg
        func_name, arg = split_func(val)
        # first validate for presence
        if func_name in presence_operations:
            func = presence_operations[func_name]
            grade = func(validation_name, current_df, arg)
            pass_col_name = "presence_pass_" + validation_name
            current_df[pass_col_name] = grade

        # then validate for correct values
        elif func_name in value_operations:
            func = value_operations[func_name]
            if validation_name in current_df.columns:
                print val, "!!"
                #grade = current_df[validation_name].apply(func, args=(validation_name, current_df, arg))
                grade = current_df.apply(func, args=(validation_name, arg, current_dm), axis=1)
                current_df["pass_" + validation_name + "_" + str(num)] = grade

  

# check that values pass validation
# required()
# requiredUnless
# min
# max
# cv (controlled vocab)
# in
# requiredIfGroup
# type
# sv (suggested vocab)



# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g

# next, do cross-contribution validations (all specimens exist, etc.)

# next, splat out each table into a file and wrap it up.  give it a sensible name.  

pass_cols = current_df.columns.str.match("^pass_")
present_cols = current_df.columns.str.match("^presen")

#current_df[pass_cols]
current_df[current_df.columns[present_cols]]
#current_df
#current_dm[['validations']][current_dm[['validations']].index.str.match("lat")]

min("age_low") !!
max("age_high") !!
cv("age_unit") !!
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Years Cal AD (+/-) is not in age_unit
returning  Yea

,presence_pass_age,presence_pass_age_high,presence_pass_age_low,presence_pass_age_unit,presence_pass_aniso_tilt_correction,presence_pass_aniso_type,presence_pass_citations,presence_pass_dir_dec,presence_pass_dir_inc,presence_pass_dir_tilt_correction,...,presence_pass_method_codes,presence_pass_paleo_lat,presence_pass_paleo_lon,presence_pass_result_quality,presence_pass_result_type,presence_pass_site,presence_pass_vadm,presence_pass_vdm,presence_pass_vgp_lat,presence_pass_vgp_lon
site,,,,,,,,,,,,,,,,,,,,,
hz05,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz06,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz07,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz09,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz10,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz11,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz13,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz15,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True
hz18,True,True,True,True,True,True,True,True,True,True,...,False,True,True,False,False,True,True,True,True,True


In [6]:
current_df[current_df.columns[pass_cols]]

,pass_age_high_0,pass_age_low_0,pass_age_unit_0,pass_geologic_classes_0,pass_geologic_types_0,pass_lat_0,pass_lat_1,pass_lithologies_0,pass_lon_0,pass_lon_1
site,,,,,,,,,,
hz05,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz06,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz07,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz09,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz10,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz11,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz13,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz15,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
hz18,True,True,Years Cal AD (+/-) is not in age_unit,Archeologic is not in class,Mixed Archeological Objects is not in type,True,True,Not Specified is not in lithology,True,True
